In [2]:
import numpy as np
from __future__ import division
import random
from keras.callbacks import History 
from keras.layers import Dense, Activation, Flatten, Conv2D, Lambda
from keras.layers import MaxPooling2D, Dropout, BatchNormalization
#from keras.utils import print_summary
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import keras.backend as K
import pickle
import csv
import cv2
import os
import numpy as np
import scipy
import pickle
import matplotlib.pyplot as plt
from itertools import islice
import warnings
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import load_model
import keras.utils

#For Visualizations
from scipy.stats import norm, skew
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
def image_preprocessing(img):
    resized_image = cv2.resize((cv2.cvtColor(img, cv2.COLOR_RGB2HSV))[:, :, 1], (40, 40))
    return resized_image

In [4]:
def Load_training(delta):
    logs = []
    features = []
    labels = []
    with open(labels_file, 'rt') as f:
        reader = csv.reader(f)
        for line in reader:
            logs.append(line)
        log_labels = logs.pop(0)

    for i in range(len(logs)):
        for j in range(3):
            img_path = logs[i][j]
            img_path = features_directory + 'IMG' + (img_path.split('IMG')[1]).strip()
            img = plt.imread(img_path)
            features.append(image_preprocessing(img))
            if j == 0:
                labels.append(float(logs[i][3]))
            elif j == 1:
                labels.append(float(logs[i][3]) + delta)
            else:
                labels.append(float(logs[i][3]) - delta)
    return features, labels

In [ ]:
features.shape

labels.shape

In [5]:
def keras_model():
    
    model = Sequential()
    model.add(Lambda(lambda x: x / 127.5 - 1., input_shape=(40, 40, 1)))

    model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(Activation('relu'))
    
    model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), padding='valid'))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), padding='valid'))

    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), padding='valid'))
    
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), padding='valid'))

    model.add(Flatten())
    model.add(Dropout(0.5))
    
    model.add(BatchNormalization())
    model.add(Dense(512))
    model.add(Dense(256))
    model.add(Dense(64))
    model.add(Dense(1))
    
    model.compile(optimizer=Adam(lr=0.0001), loss="mse")
    model.summary()
    return model

In [ ]:
def loadFromPickle():
    with open("features", "rb") as f:
        features = np.array(pickle.load(f))
    with open("labels", "rb") as f:
        labels = np.array(pickle.load(f))
    return features, labels

def augmentData(features, labels):
    features = np.append(features, features[:, :, ::-1], axis=0)
    labels = np.append(labels, -labels, axis=0)
    return features, labels

In [ ]:
warnings.filterwarnings('ignore')

features, labels = loadFromPickle()
features, labels = shuffle(features, labels)

#do a Train Test Split
train_x, val_x, train_y, val_y = train_test_split(features, labels, random_state=42, test_size=0.20)

train_x = train_x.reshape(train_x.shape[0], 40, 40, 1)
val_x = val_x.reshape(val_x.shape[0], 40, 40, 1)

model = keras_model()
filepath = "./SelfDriving.h5"

es = EarlyStopping(monitor='val_loss', verbose=1)
checkpoint = ModelCheckpoint(filepath, verbose=1,monitor='val_loss', save_best_only=True)
hist = History()
callbacks_list = [es,checkpoint,hist]
    
model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=10, batch_size=32,callbacks=callbacks_list)

print_summary(model)

model.save(filepath)

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()